In [ ]:
import panel as pn

import pandas as pd
import holoviews as hv

from sklearn.cluster import KMeans

pn.extension('ace', sizing_mode="stretch_width")

import hvplot.pandas

## Slideshow

In [ ]:
slider = pn.widgets.IntSlider(start=0, end=42, max_width=900)

img = pn.pane.GIF("https://raw.githubusercontent.com/MarcSkovMadsen/awesome-panel-assets/master/panel/slideshow/0.gif", embed=False, height=400, sizing_mode="scale_height")

slider.jscallback(args={'img': img}, value="""
img.text = '<img src="https://raw.githubusercontent.com/MarcSkovMadsen/awesome-panel-assets/master/panel/slideshow/'+cb_obj.value+'.gif" style="height:100%;object-fit: cover"></img>'
""")

app = pn.Column(slider, img, name="Output", height=450)

ace = pn.widgets.Ace(readonly=True, height=500, language='python', value=\
"""\
import panel as pn

pn.extension(sizing_mode="stretch_width")

slider = pn.widgets.IntSlider(start=0, end=42, max_width=800)

def slideshow(index):
    url = f"https://raw.githubusercontent.com/MarcSkovMadsen/awesome-panel-assets/master/panel/slideshow/{index}.gif"
    return pn.pane.GIF(url, embed=False, sizing_mode="scale_height", height=400, max_width=800)

output = pn.bind(slideshow, slider)

app = pn.Column(slider, output)
app.servable()""", name="Code")

app1 = pn.Tabs(app, ace)

## Penguins

In [ ]:
penguins = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv').dropna()
cols = list(penguins.columns)[2:6]

x = pn.widgets.Select(name='x', options=cols)
y = pn.widgets.Select(name='y', options=cols, value='bill_depth_mm')
n_clusters = pn.widgets.IntSlider(name='n_clusters', start=2, end=5, value=3)

def cluster(data, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters)
    est = kmeans.fit(data)
    return est.labels_.astype('str')

def plot(x, y, n_clusters):
    penguins['labels'] = cluster(penguins.iloc[:, 2:6].values, n_clusters)
    penguins_plot = penguins.sort_values('labels').hvplot.scatter(
        x, y, c='labels', hover_cols=['species'], line_width=1, size=100, responsive=True
    ).opts(marker=hv.dim('species').categorize({'Adelie': 'square', 'Chinstrap': 'circle', 'Gentoo': 'triangle'}))
    
    centers = penguins.groupby('labels').mean()
    centers_plot = centers.hvplot.scatter(
        x, y, marker='x', color='black', padding=0.1, line_width=5, responsive=True, size=400
    )         
                     
    return  penguins_plot * centers_plot

explanation = pn.pane.Markdown("""
This app applies **k-means clustering on the Palmer Penguins dataset** using scikit-learn, parameterizing the number of clusters and the variables to plot.
<br><br>
Each cluster is denoted by one color while the penguin species is indicated using markers: 
<br><br>
● - Adelie, ▲ - Gentoo, ■ - Chinstrap""", margin=(10,15))

code = pn.widgets.Ace(language='python', height=360, value=\
"""import holoviews as hv
import hvplot.pandas
import pandas as pd
from sklearn.cluster import KMeans

import panel as pn

pn.extension(sizing_mode="stretch_width")

penguins = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv').dropna()

def cluster(data, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters)
    est = kmeans.fit(data)
    return est.labels_.astype('str')

def plot(x, y, n_clusters):
    penguins['labels'] = cluster(penguins.iloc[:, 2:6].values, n_clusters)
    penguins_plot = penguins.sort_values('labels').hvplot.scatter(
        x, y, c='labels', hover_cols=['species'], line_width=1, size=100, responsive=True
    ).opts(marker=hv.dim('species').categorize({'Adelie': 'square', 'Chinstrap': 'circle', 'Gentoo': 'triangle'}))
    
    centers = penguins.groupby('labels').mean()
    centers_plot = centers.hvplot.scatter(
        x, y, marker='x', color='black', padding=0.1, line_width=5, responsive=True, size=400
    )         
                     
    return  penguins_plot * centers_plot

cols = list(penguins.columns)[2:6]
x = pn.widgets.Select(name='x', options=cols)
y = pn.widgets.Select(name='y', options=cols, value='bill_depth_mm')
n_clusters = pn.widgets.IntSlider(name='n_clusters', start=2, end=5, value=3)

explanation = pn.pane.Markdown('''
This app applies **k-means clustering on the Palmer Penguins dataset** using scikit-learn, parameterizing the number of clusters and the variables to plot.
<br><br>
Each cluster is denoted by one color while the penguin species is indicated using markers: 
<br><br>
● - Adelie, ▲ - Gentoo, ■ - Chinstrap
''', margin=(10,15))

pn.Row(
    pn.WidgetBox(x, y, n_clusters, explanation, width=250, sizing_mode="fixed"),  
    pn.panel(pn.bind(plot, x, y, n_clusters), sizing_mode="stretch_both"),
).servable()""", name="Code")

app2 = pn.Tabs(
    pn.Row(
        pn.WidgetBox(x, y, n_clusters, explanation, width=250, sizing_mode="fixed"),  
        pn.panel(pn.bind(plot, x, y, n_clusters), sizing_mode="stretch_both"),
        name="Output"),
    code
)

In [ ]:
pn.Tabs(
  ('Slideshow', app1),
  ('Penguin K-Means Clustering', app2),
).servable() #.embed(max_opts=4, json=True, json_prefix='json')